In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
plt.style.use('ggplot')
pd.set_option('display.max_columns', None) # to display all columns of each printed dataframe

# Loading Data

The data on schools in England come from the UK government, specifically from the school performance comparison service:
https://bit.ly/2YKztfa

The current dataset includes only records for the latest available school year, 2018-2019, and only for *primary* schools (key stages 1 and 2, for children of age 5-11, though some schools in this dataset also have students of older age). 

In [2]:
%%time
df = pd.read_excel('england_ks2final.xlsx', na_values="SUPP")

Wall time: 58 s


The file is quite large and takes about a minute to load

In [3]:
df.memory_usage().sum() # bytes of memory consumed by the whole dataframe

41996480

Let's take a look at the dataframe dimensionality, feature names and types

In [4]:
df.shape

(16508, 318)

In [5]:
df.columns

Index(['RECTYPE', 'ALPHAIND', 'LEA', 'ESTAB', 'URN', 'SCHNAME', 'ADDRESS1',
       'ADDRESS2', 'ADDRESS3', 'TOWN',
       ...
       'MATPROG_UNADJUSTED', 'READPROG_DESCR_17', 'WRITPROG_DESCR_17',
       'MATPROG_DESCR_17', 'READPROG_DESCR_18', 'WRITPROG_DESCR_18',
       'MATPROG_DESCR_18', 'READPROG_DESCR', 'WRITPROG_DESCR',
       'MATPROG_DESCR'],
      dtype='object', length=318)

In [6]:
df.index # uses pandas default index, for now

RangeIndex(start=0, stop=16508, step=1)

In [7]:
df.index.is_unique

True

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16508 entries, 0 to 16507
Columns: 318 entries, RECTYPE to MATPROG_DESCR
dtypes: float64(200), int64(1), object(117)
memory usage: 40.1+ MB


# Data Cleaning and Preprocessing

## Feature Selection

The total number of features (columns) in the dataframe is quite large, at 318. Investigating all of these is unlikely to be feasible, and we can instead focus only on a subset of these, with the meanings as in the table below.

To a large extent, feature selection for the purpose of this project is a judgement call, since pretty much all of them contain useful information. Some of the reasons are quite straightforward: certian columns (like school phone number) are not relevant for this type of analysis, and many columns contain duplicated information and/or are unambiguously redundant (e.g. the *number* of pupils achieving high proficiency in mathematics vs the *percentage* of pupils achieving the same, or the percentages of *disadvantaged* pupils in the same category vs the percentage of *non-disadvantaged* pupils in the same category). Since the focus will be on school performance as a whole, features pertaining specifically to boys or girls will not be included either (moreover, proper accounting for them would be complicated by the fact that a significant proportion of schools in England are single-sex, and thus a girls-only school will have N/A for all features pertaining to boys). 

The decision was also made *not* to include any features containing raw *scores* in any subject or category, for now, and focus instead on *percentages*. One of the reasons for this is the intention to apply unsupervised learning methods to explore the data further, and some of these methods (e.g. K-Means clustering) require features to be on the same scale. While it is generally easy to rescale them (using, for example, sklearn.preprocessing.StandardScaler) doing so might introduce additional inaccuracy to data, and will be left aside, for now. Retained continuous features are all expected to have the same scale (percentage), but this will need to be checked later. 

Finally, several categorical features will be included, including, among others, religious denomination and school type. While some of these columns may contain similar information (specifically postcodes, town names, and parliamentary constituencies) they will be retained for now. They are relevant for analysis (it will be important to undestand, for example, whether schools with religious affiliation tend to perform better or worse than more 'secular' ones, and whether this difference, if any, is statistically significant). Since the main goal of the project is to identify locations with better schools, several approaches to 'slicing' the data (e.g. by towns and post districts) will be attempted later. 

|Field Name|Description|
|:--|:--|
|URN|School unique reference number|
|SCHNAME|School/Local authority name|
|TOWN|School town|
|PCODE|School postcode|
|PCON_NAME|School parliamentary constituency name|
|NFTYPE|School type. Rectype 1: AC = Academy Sponsor Led (NFTYPE 20), CY = Community school (21), VA = Voluntary Aided school (22), VC = Voluntary Controlled school (23), FD = Foundation school (24), CTC = City Technology College (25), ACC = Academy converter (51), F = Free school (52). Rectype 2: CYS = Community special (26), FDS = Foundation special (27), ACS = Academy special (50), FS = Free Special School (53), ACCS = Academy Converter Special (55)|
|RELDENOM|Religious denomination|
|AGERANGE|Age range|
|PTKS1GROUP_L|Percentage of pupils in cohort with low KS1 attainment|
|PTKS1GROUP_M|Percentage of pupils in cohort with medium KS1 attainment|
|PTKS1GROUP_H|Percentage of pupils in cohort with high KS1 attainment|
|PTNotFSM6CLA1A|Percentage of key stage 2 pupils who are not disadvantaged|
|PTRWM_EXP|Percentage of pupils reaching the expected standard in reading, writing and maths|
|PTRWM_HIGH|Percentage of pupils achieving a high score in reading and maths and working at greater depth in writing|
|PTREAD_EXP|Percentage of pupils reaching the expected standard in reading|
|PTREAD_HIGH|Percentage of pupils achieving a high score in reading|
|PTGPS_EXP|Percentage of pupils reaching the expected standard in grammar, punctuation and spelling|
|PTGPS_HIGH|Percentage of pupils achieving a high score in grammar, punctuation and spelling|
|PTMAT_EXP|Percentage of pupils reaching the expected standard in maths|
|PTMAT_HIGH|Percentage of pupils achieving a high score in maths|
|PTWRITTA_EXP|Percentage of pupils reaching the expected standard in writing|
|PTWRITTA_HIGH|Percentage of pupils working at greater depth within the expected standard in writing|
|PTSCITA_EXP|Percentage of pupils reaching the expected standard in science TA|
|PTEALGRP1|Percentage of eligible pupils with English as first language|
|PSENELE|Percentage of eligible pupils with EHC plan|
|PSENELK|Percentage of eligible pupils with SEN support|
|PTNOTFSM6CLA1A_18|Percentage of key stage 2 pupils who are not disadvantaged one year prior|
|PTRWM_EXP_18|Percentage of pupils reaching the expected standard in reading, writing and maths one year prior|
|PTRWM_HIGH_18|Percentage of pupils achieving a high score in reading and maths and working at greater depth in writing  one year prior|
|PTNOTFSM6CLA1A_17|Percentage of key stage 2 pupils who are not disadvantaged two years prior|
|PTRWM_EXP_17|Percentage of pupils reaching the expected standard in reading, writing and maths two years prior|
|PTRWM_HIGH_17|Percentage of pupils achieving a high score in reading and maths and working at greater depth in writing two years prior|
|PTRWM_EXP_3YR|Percentage of pupils reaching the expected standard in reading, writing and maths  - 3 year total|
|PTRWM_HIGH_3YR|Percentage of pupils achieving a high score in reading and maths and working at greater depth in writing  - 3 year total|


In [ ]:
selected_columns = [
    "URN", "SCHNAME", "TOWN", "PCODE", "PCON_NAME", "NFTYPE", "RELDENOM", "AGERANGE", 
    "PTKS1GROUP_L", "PTKS1GROUP_M", "PTKS1GROUP_H", "PTNotFSM6CLA1A",
    "PTRWM_EXP", "PTRWM_HIGH", "PTREAD_EXP", "PTREAD_HIGH", "PTGPS_EXP", 
    "PTGPS_HIGH", "PTMAT_EXP", "PTMAT_HIGH", "PTWRITTA_EXP", "PTWRITTA_HIGH", 
    "PTSCITA_EXP", "PTEALGRP1", "PSENELE", "PSENELK", "PTNOTFSM6CLA1A_18", 
    "PTRWM_EXP_18", "PTRWM_HIGH_18", "PTNOTFSM6CLA1A_17", "PTRWM_EXP_17", 
    "PTRWM_HIGH_17", "PTRWM_EXP_3YR", "PTRWM_HIGH_3YR"
    ]
df = df[selected_columns]

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.memory_usage().sum()

## Duplicated or Missing Values

In [ ]:
df.duplicated().values.any()

All records (rows) in the dataframe are unique

In [ ]:
df.isnull().values.any()

However, some of these records contain missing values

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df.info()

Now only 34 features are used, instead of previous 318. Memory usage also went down from more than 40 MB to around 4 MB.

Looking at data types, we can notice that continuous features are all of type float64 (consistent with the fact that they are all supposed to be expressed as percentages), while the categorical ones (e.g. the names of parliamentary constituencies) are of type object. The latter is expected too, since object is supposed to imply text or mixed numeric and non-numeric values. 

Also we can see that the number of non-null values is *not* the same across features. This may also suggest a problem with missing data, which has to be dealt with accordingly.

Let's take a look at first 10 and last 10 rows of the dataframe.

In [ ]:
df.head(10)

In [ ]:
df.tail(10)

We can see an inconsistency here. While most *records* (or rows) in the dataframe refer to individual schools, the bottom ones appear to be aggregates by boroughs or counties, as well as totals. 

Let's move the records which do *not* have URN (unique reference number) and hence do *not* correspond to individual schools, into a separate dataframe. 

Inconveniently, the missing URNs are not always blank (these are usually automatically recognised as N/A), but are more often specified as empty stings with whitespace (e.g. " "). This can also explain why we had '16508 non-null  object' for URN (if all missing ones were properly coded as N/A, there would have been less). 

In [ ]:
# df[df['URN'].isna()]
df = df.dropna(subset=['SCHNAME'])
df = df[df['SCHNAME'] != " "] # removing the last row with empty school name
df_bor = df[(df['URN'].str.contains(' ') == True) | (df['URN'].isna())]
df_bor['URN'] = np.nan # populating all school URNs for boroughs with N/A for consistency

In [ ]:
df = df[df['URN'].notna()]
df = df[~(df['URN'].str.contains(' ') == True)] # df should now contain only individual schools

In [ ]:
df_bor.head(10)

In [ ]:
df_bor.tail(10)

In [ ]:
df.head(10)

In [ ]:
df.tail(10)

In [ ]:
df['URN'].isna().any()

No N/A in the main dataframe, for unique reference numbers

In [ ]:
df_bor['URN'].isna().all()

The dataframe for *boroughs* contains N/A for school URNs, as intended

In [ ]:
df.shape

We can see there are 16355 Englsih schools altogether in the schools dataframe

In [ ]:
df.describe()

We can see that the scale of each continuous feature is indeed from 0 to 1, as expected (since they all are percentages), so we do not have obvious errors or outliers in the data, at least from first glance. But this will need to be checked further. 

Note that the number of columns in the table above is 26. The describe method of the pandas.DataFrame class includes, by default, only numeric and object features. The total number of features in the dataframe is 34. This is expected, since the first 7 features are distinctly categorical (e.g. religious denomination). 

Nevertheless, the fact that descriptive statistics were calculated for all features where we expected them does *not* mean on its own that everything is fine with these columns. Specifically we will need to check for N/A values, which are generally ignored in calculations. 

In [ ]:
df.index # still the default pandas index

It will be more convenient if we change the default index to one of the columns, specifically to URN (only in the dataframe containing only school records, each with its own unique reference number)

In [ ]:
df.set_index('URN', inplace=True)
df.index

In [ ]:
df.index.is_unique

This worked fine. Our index is of type Int64Index, which, as the name suggests, allows only for integers, implying that the issue we had with missing values represented as spaces or empty strings has been resolved. Now every index is a series of URNs, each is recognised as integer as expected, and every single value in the index series is unique. 

Let's explore these missing values in more detail. 

In [ ]:
len(df[df.isna().any(axis=1)]) / len(df)

Almost 17% of all rows contain at least one N/A in at least one of their 33 columns. 
We may want to impute them later, but first need to understand why exactly they are missing. 

Imputation is generally complicated, and in many cases it is actually not needed, particularly for cases where data are missing *not* at random

In [ ]:
df.shape

In [ ]:
df.count().sort_values(ascending=False)

Let's start with easier ones and figure out why one school has no TOWN and two schools have no PCON_NAME (parliamentary constituency). 

In [ ]:
df[df['TOWN'].isnull()]

Looks like just a genuine omission for no good reason. This one is easy to fix, but we need to know what to replace it with (should it be Rye, Rye Harbour, RYE or something else?).

Let's check some other schools with similar postcodes.

In [ ]:
df[(df['PCODE'].str.contains('TN31')) & (df['PCON_NAME']=='Hastings and Rye')]

This is strange. Turns out there are *two* schools not only in the exact same postcode, but sharing the same name: both are called Rye Community Primary School. Could it be the case that this is actually an error and the same school was included twice? This is entirely possible. However, we can see that URNs are actually different, suggesting these may be two different legal entities altogether.

Turns out that is indeed the case: one school was replaced by another, and URN 146826 actually refers to the new legal entity, which was established in 2018: https://get-information-schools.service.gov.uk/Establishments/Establishment/Details/141807#school-links

This should also explain why so many features are missing for this school. 

Should we delete one of these entities or keep both? Since they refer to the same school, in effect, it makes sense to leave only one. Which one to choose? In this case, it may be more helpful to keep the 'old' one, even if it technically no longer exist, since it has more historical data available. Since the aim of this project is not to select only one school but rather to select a good area, keeping a few defunct entities might be justified. It may be the case that the data available for that nonexisting school will help us learn more about the eare of Hastings and Rye. 

Let's update the TOWN column and delete the first row.

In [ ]:
df.at[141807, 'TOWN'] = 'Rye'

In [ ]:
df.drop(index=146826, inplace=True)

In [ ]:
df.at[141807, 'TOWN']

In [ ]:
df[(df['PCODE'].str.contains('TN31')) & (df['PCON_NAME']=='Hastings and Rye')] # no longer any N/A under TOWN, and now only three rows instead of four as before

The fact that we have found at least one defunct school also brings about another important topic tangentially related to the topic of data cleaning in data science: *survivorship bias*. It is highly likely that the database includes only schools (with a few exceptions) which are still operational, and thus the schools which failed or were closed for other reasons are not going to be found in it. Since our main goal is to understand which areas in England have better schools, this may imply that potentially a large chunk of important data is missing in our analysis, and we may consider looking for it separately (e.g. there may be a table in the same source containing exclusively schools which were closed at some point in the past). 

Overall, identifying and managing duplicates is an important step in exploratory data analysis. This is something we need to do even before any algorithms are applied (because, obviously, duplicates may cause some schools to be counted more than once, thus potentially skewing any result). 

First of all, let's check to see if there are any rows with *all* features duplicated (none are expected)

In [ ]:
df[df.duplicated()] 

As we can see, every row is technically unique (even if the difference between two rows is in only one column out of 33). 

Now that we have established that there are differences in rows, let's check whether there are any other schools with identical names (which may indicate that they refer to the same school, but not necessarily so)

In [ ]:
sch_name_count = df['SCHNAME'].value_counts()
sch_name_count[sch_name_count > 1]

In [ ]:
len(sch_name_count)

In [ ]:
sum(sch_name_count)

There are 16354 English schools altogether in the database. This is consistent with the figure we found earlier (remember that one duplicate has been already removed)

In [ ]:
sch_names_not_unique = sch_name_count[sch_name_count > 1].index.tolist()
len(sch_names_not_unique)

In [ ]:
sch_names_not_unique[:10]

We can see that there is a large number of schools with non-unique names. Some of these are likely to be genuinely different schools (perhaps mostly the ones with religious affiliation), but we should to check it. 

One way to do it will be to get all non-unique school names into a list and apply a filter on the main dataframe to show *only* the schools in this list. The first part is already done, the list is exactly the one we looked at one line above. Now let's apply a filter and get a slice of a dataframe containing *only* schools with non-unique names. 

In [ ]:
df[df.SCHNAME.isin(sch_names_not_unique)].sort_values(by='SCHNAME')

We can see quite clearly that *not* all schools with the shared names are duplicates. The Abbey Primary Schools above, for example, clearly refer to different entities altogether, as they all are based in different cities. However, what would also be helpful is to see whether there are any schools which share the same name and *postcode*, since these should in theory be based in the same building, and may well be the same schools. 

In [ ]:
sch_pcode_count = df['PCODE'].value_counts()
sch_pcodes_not_unique = sch_pcode_count[sch_pcode_count > 1].index.tolist() # postcodes which occur more than once in the dataframe
df[df.PCODE.isin(sch_pcodes_not_unique)][df.SCHNAME.isin(sch_names_not_unique)].sort_values(by='SCHNAME')

This seems to be quite important. We have found 266 schools where *both* school names and postcodes are not unique. The few rows we can see above clearly demonstrate that the situation with Rye Community Primary School was not really a special case. We can see something similar for a fairly large number of other schools as well. 

Let's check the first two schools in the list. It is indeed the case that they are related: https://get-information-schools.service.gov.uk/Establishments/Establishment/Details/119460#school-links

How are we going to handle this?  Clearly samples (rows) where most features (columns) are N/A are not really useful to us. For the same reasons as outlined above, we can keep the *predecessor* schools and remove the successor ones (which also tend to have large numbers of N/A values). Clearly we don't want to remove successor schools if they have sufficient data populated. To work around this problem, let's drop all rows which contain too many N/A, above the relatively high threshold of 26 (we can see above that our dataframe has 33 columns, and hence we are going to drop all rows which *don't* have at least 26 non-N/A values). The number 26 comes from the fact that the first 7 features are categorical (e.g. town, religious denomination). It is entirely possible, as we can see above, for the new school to have first 7 columns populated, but have no values for numerical features at all. Since numerical features are the ones in which we are mostly interested, let's remove the rows where they are missing. 

In [ ]:
df.shape # dataframe shape before N/A removal

In [ ]:
df = df.dropna(thresh=26)

In [ ]:
df.shape # dataframe shape after N/A removal

The decrease in size is quite large: more than 1000 schools had at least 25 N/A values out of total 33. 

Let's check, once again, whether there are still any schools where both the school name and the postcode are not unique. 

In [ ]:
df[df.duplicated(subset=['SCHNAME','PCODE'], keep=False)]

## Special Schools Treatment

The code above has worked as expected: we no longer have any schools which share the same name and postcode. 

However, while working on cleaning N/A values, one may have noticed that certain rows contain a suspiciously large number of zeros. Are these also indicative of missing values? We need to take a closer look. 

The specific columns where zeros look particularly unexpected are PTRWM_EXP, PTRWM_EXP_18, and PTRWM_EXP_17 (percentage of pupils reaching the expected standard in reading, writing and maths in 2019, 2018, and 2017 respectively).

In [ ]:
subset_columns = ["SCHNAME", "TOWN", "NFTYPE", "RELDENOM", "AGERANGE", "PTKS1GROUP_L", "PTKS1GROUP_M", "PTKS1GROUP_H", "PTRWM_EXP", "PSENELE", "PSENELK", "PTRWM_EXP_18", "PTRWM_EXP_17", "PTRWM_EXP_3YR"]
df_special = df[(df.PTRWM_EXP == 0) & (df.PTRWM_EXP_18 == 0) & (df.PTRWM_EXP_17 == 0)][subset_columns].sort_values(by='PTKS1GROUP_L', ascending=False) # sorted by percentage of pupils with low KS1 attainment
df_special

In [ ]:
df_special.PTKS1GROUP_L.value_counts(bins=10) # Percentage bins of low results for key stage 1 tests 

Interesting. There is clearly a pattern here. We can see that there is a fairly large number of schools where **none** of the students managed to achieve the expected standard in reading, writing and maths within the last three years.

Are these errors in the data? Most likely no. We can also see that for the overwhelming majority of these schools more than 90% of their students have achieved low KS1 attainment (the overall average, as we can see below, was around 88%, while only around 10% achieved medium).

Are these really bad schools? Almost certainly no. The table below also highlights another important observation: more than 98% of these students had *EHC plans* (column PSENELE). In England, an Education, Health and Care (EHC) plan implies children with special educational needs. Beaucroft Foundation School, the first school in the table above, for example, is a school for children with an Autistic Spectrum Disorder. Likewise, Mount Tamar School specialises "in supporting students with social and emotional difficulties, attachment and autism".

We can see below that, in 385 schools out of 410, more than 90% of students have EHC plans. 

In [ ]:
df_special.PSENELE.value_counts(bins=10)

In [ ]:
df_special.describe()

Clearly these schools are in a different category of their own (special education) and hence are not exactly comparable with most other schools in the dataframe. Let's keep them in a different dataframe, df_special, and remove from the main one. 

In this case specifically we are going to remove all schools where at least half of students have EHC plans. 

In [ ]:
df = df[df.PSENELE <= 0.5]

When we run describe() again, the results look more reasonable. However, there are still schools which are likely to be underperforming (with none of the students achieving the expected level of reading and writing proficiency), and we need to look at these. 

In [ ]:
df[subset_columns].describe() # columns are still scaled from 0 to 1, as expected; no outliers

In [ ]:
df[df.PTRWM_EXP < 0.1][subset_columns] # Schools with less than 10% of students reaching the expected standard in reading, writing and maths

In [ ]:
len(df[df.PTRWM_EXP < 0.1])

As we can see above, there is only 21 such school in England (*not* counting schools for children with special educational needs). Even for these schools, however, the distribution of key stage 1 results (PTKS1GROUP: low, medium, and high) does not seem to be abnormal (most students in each are in the medium band). Why are there no students then achieving the expected level of reading, writing, and maths (PTRWM_EXP)? There is no obviousl answer to this question. Possibly at least some of these schools are new and have no children which have already been *tested* on these skills (they may actually refer to Key Stage 2, unlike PTKS1GROUP, which is for Key Stage 1). We may need to take a closer look at this, but given that there are only 21 schools like that out of 14560, they are unlikely to make much of a difference. 

Some spot checks suggest that these schools are about as normal as one can expect. The first one, for example (Chaddleworth St Andrew's C.E. Primary School), is actually rated Good by the Office for Standards in Education, Children's Services and Skills (i.e. school inspectors). 

Let's leave them for now and take a look at the dataframe as a whole. 

In [ ]:
df.info()

It is slightly unusual that in columns 0 to 24 inclusive the non-null counts are not the same. This implies that there are still some N/A values which we need to check. 

From column 25, differences can be justified, since they implicitly refer to the data from *prior* years, and not all schools may have prior history available (for example, they may be just new). 

Let's focus on the columns from 0 to 24 inclusive and check all rows containing at least one N/A. Is it the same school?

In [ ]:
df.iloc[:, :25][df.isna().any(axis=1)]

As we can see, the number is fairly large. 951 schools in total have at least one N/A. 

Let's check a few columns closer. 

In [ ]:
df.iloc[:, :25][df.isna().any(axis=1)][["SCHNAME", "TOWN", "NFTYPE", "RELDENOM", "AGERANGE", "PTKS1GROUP_L", "PTKS1GROUP_M", "PTKS1GROUP_H", "PTRWM_EXP", "PSENELE", "PSENELK"]].head(30)

In [ ]:
df[df.PTGPS_EXP.isnull()] # Percentage of pupils reaching the expected standard in grammar, punctuation and spelling

In [ ]:
df[df.PTMAT_EXP.isnull()] # Percentage of pupils reaching the expected standard in maths

In [ ]:
df[df.PTRWM_EXP.isnull()] # Percentage of pupils reaching the expected standard in reading, writing and maths

While there are indeed some gaps for several schools, including the two above, there likely is no strong case for excluding them, since much of the data are actually populated. 

However, we need to be careful later, particularly so while performing clustering analysis. One problem with some algorithms is that they may exclude *any* samples which have at least one feature missing. Before clustering (which will likely be done only on a subset of features) we need to check for missing data again, to ensure no relevant data are excluded unintentionally. 

## School Comparison by Religious Denomination

Now that we have addressed the issue of missing data, we can take a look at religious denominations. This is also going to be important for our analysis, since one of the questions we are trying to answer is whether religious schools tend to be any better, statistically, than non-religious one (and if yes, which exactly). 

In [ ]:
df.RELDENOM.unique()

Let's take a look at the breakdown of schools by religious denomination. 

In [ ]:
df.RELDENOM.value_counts()

In [ ]:
df.RELDENOM.value_counts(normalize=True)

In [ ]:
reldenom_percentages = df.RELDENOM.value_counts(normalize=True)*100
reldenom_percentages_short = pd.concat([reldenom_percentages[:'Roman Catholic'], pd.Series({'Other':reldenom_percentages['None':].sum()})]) # Replacing schools from None inclusive by their sum
reldenom_percentages_short.plot(kind='bar', title='Schools by Religious Denomination')

There may be a problem here: 'Does not apply' (and potentially 'None') and 'Unknown' may refer to the same thing, but coded in a different way. Let's take a closer look at these. 

In [ ]:
df[df.RELDENOM.isin(['Does not apply', 'None', 'Unknown'])].head(20)

A brief look at the schools with 'Does not apply' under the feature corresponding to their religious denomination suggests that there might be a reason for it. Could it be the case that only schools for *older* children (i.e. non-primary) can have religious denomination? Possibly, but that doesn't seem likely from what we have seen already. Since we are going to compare directly religious and non-religious schools, we need to understand whether they are comparable. 

Let's start by taking a look at age distributions first, between 'Does not apply' (the largest secular category) and 'Church of England' (the largest religious category). Later, we should conduct a more detailed analysis and visualisation, but this is going to be important to get some first ideas. 

In [ ]:
df[df.RELDENOM == 'Does not apply'].AGERANGE.value_counts(normalize=True) # normalize for relative frequencies

In [ ]:
df[df.RELDENOM == 'Church of England'].AGERANGE.value_counts(normalize=True) # normalize for relative frequencies

While there are differences in the number of 'ranges' (possibly related to the fact that there are more schools in the former category than in the later), both seem to be overlapping, as they both include children from the age of 2 to the age of 19. This suggests that we can actually compare religious and secular schools. 

To make this comparison easier, let's add a new boolean column 'SECULAR', which will have 1 if RELDENOM is 'Does not apply', 'None', or 'Unknown' and 0 otherwise (strictly speaking, 'Unknown' would be better counted as N/A, but given that there are only 2 such schools, that is likely to be immaterial). 

There are multiple ways to add a new column to a pandas dataframe and populate its values with a condition depending on other column. A more pythonic (and potentially faster, if a vectorised function is used instead of a loop) way to do it will be to use np.where

In [ ]:
df['SECULAR'] = np.where(df['RELDENOM'].isin(['Does not apply', 'None', 'Unknown']), 1, 0)

Let's run a quick check. We expect all schools with 1 under SECULAR to have RELDENOM as one of the three above, while all religious schools will be where SECULAR is 0. 

In [ ]:
df[df['SECULAR'] == 1]['RELDENOM'].unique()

In [ ]:
df[df['SECULAR'] == 0]['RELDENOM'].unique()

In [ ]:
df['SECULAR'].unique()

We can see that this has worked exactly as expected. However, on a second thought, it will be better if we have it more explicit: 'Secular' and 'Religious' instead of 1 and 0. With this, we should also rename the column, since SECULAR suggests a boolean value (RELTYPE would be a more generic name). Both changes are quite easy to do in pandas. 

In [ ]:
df.rename(columns={'SECULAR': 'RELTYPE'}, inplace=True)
df['RELTYPE'].replace([0, 1], ['Religious', 'Secular'], inplace=True)
df['RELTYPE'].unique()

In [ ]:
df.info() # RELTYPE has 14560 non-null values, which is the same as RELDENOM, as expected

Let's take a look whether the means are any different between secular and religious schools. The expectation is that the differences are not going to be statistically significant at all. 

Of special interest will PTKS1GROUP columns (for low, medium, and high Key Stage 1 attainment levels respectively). 

It is important to note that relying only on the means can potentially be misleading (especially if outliers are present), but we will take a look at some other measures of central tendency a bit later. 

In [ ]:
df.groupby('RELTYPE').mean()

The table above is useful, but not too convenient to check, particularly across all columns. Let's visualise it using seaborn. However, before doing that we need to stack the dataframe (i.e. to convert if from 'wide' to 'long')

In [ ]:
df_secular_groups = df.groupby('RELTYPE').mean().stack()
df_secular_groups=df_secular_groups.to_frame()
df_secular_groups.reset_index(inplace=True)
df_secular_groups.columns = ['reltype', 'category', 'mean']

In [ ]:
g = sns.FacetGrid(df_secular_groups, col='category', col_wrap=4, margin_titles=True, legend_out=True, sharex=False)
g.map(sns.barplot, 'reltype', 'mean', order=['Secular', 'Religious'])

The results for KS1 attainments look comparable for both groups, and so are percentages for reading, writing, and math (PTRWM), reading (PTREAD), grammar, punctuation, and spelling (PTGPS), mathematics (PTMAT), writing (PTWRITTA), science (PTSCITA).  Comparable results can be seen for prior years' data (the rightmost columns in the table above). 

The data above are useful, but the comparison above was quite high-level. Specifically, it was done only for the *means* of the features, which may have been affected by outliers (the main suspect is PTNotFSM6CLA1A, percentage of children who are not disadvantaged, where we *do* see an apparent difference between religious and non-religious schools). Can we take a look a other measures of central tendency, and perhaps even visualise the outliers, if any? We certainly do, and the way to do it is to use box plots. Likewise, let's convert the dataframe from wide to long format by stacking, and use seaborn afterwards. 

In [ ]:
cols = df.groupby('RELTYPE').mean().columns # to get the same (numerical) columns as in the table above
df_long = df[cols].stack()
df_long=df_long.to_frame()
df_long.reset_index(inplace=True)
df_long.columns = ['urn', 'category', 'value']
df_long['reltype'] = np.where(df_long['urn'].isin(df[df['RELTYPE']=='Secular'].index), 'Secular', 'Religious')

In [ ]:
set(df[df['RELTYPE']=='Secular'].index) - set(df_long[df_long['reltype']=='Secular']['urn']) # a quick check to ensure that secular columns contain the same unique values

In [ ]:
set(df[df['RELTYPE']=='Religious'].index) - set(df_long[df_long['reltype']=='Religious']['urn'])

In [ ]:
g = sns.FacetGrid(df_long, col='category', col_wrap=4, margin_titles=True, legend_out=True, sharex=False)
g.map(sns.boxplot, 'reltype', 'value', order=['Secular', 'Religious'])

Let us remind what the boxes in the plots above actually represent. The boxes visualise the distributions across categories, for secular and religious schools (the left box and right box respectively). The horizontal lines in the middle of these boxes are medians (which, unlike means, are not affected by outliers). The box itself shows four quartiles, with its borders standing for first and third quartiles. The whiskers, in addition to box borders, stand for minimum and maximum, and outliers are shown as individual black dots. 

We can, however, take this one step further and also plot the *distributions* across different categories. This will require a different type of a plot, the violin plot specifically. However, before we go there, let's run summary statistics across the two categories, to show the data in the tabular form first. 

In [ ]:
df[df.RELTYPE == 'Religious'].describe()

In [ ]:
df[df.RELTYPE == 'Secular'].describe()

Now let's show this using a similar grid, but this time containing violin plots. 

In [ ]:
g = sns.FacetGrid(df_long, col='category', col_wrap=4, margin_titles=True, legend_out=True, sharex=False)
g.map(sns.violinplot, 'reltype', 'value', split=True, order=['Secular', 'Religious'])

We can see in the grids above that most figures (both 'boxes' and 'violins') are comparable, across most categories, between secular and religious schools. Where we *do* see an apparent difference is in statistics for key stage 2 children (age 7-11) who are not disadvantaged (PTNotFSM6CLA1A, in the top right corner). For these columns (including prior year's data), there is a difference which seems to be too large to be a chance. Why is it different? There is no obvious answer yet, but it may have something to do with outliers (we can see more black dots in the top-right chart for religious schools in the box plot, and also a longer lower 'tail' in the violin plot), and we need to take a closer look there. 

In [ ]:
df[df.RELTYPE == 'Secular'].sort_values(by='PTNotFSM6CLA1A', ascending=True)[['SCHNAME', 'TOWN', 'NFTYPE', 'RELDENOM', 'AGERANGE', 'PTNotFSM6CLA1A']]

We can see in the table above that there are certain schools where only 4-5% of children are *not* disadvantaged, which are also secular (and are community schools, as suggested by CY under NFTYPE). Let's take a look at the top two, Marshland Primary School and Leasowe Primary School. A quick glance on their details from the government website suggests that this is correct. We can also see that in both schools there is quite a large percentage of students who are eligible for free school meals, another indicator of potential disadvantages. 

https://www.get-information-schools.service.gov.uk/Establishments/Establishment/Details/117937

https://www.get-information-schools.service.gov.uk/Establishments/Establishment/Details/105055

Let's see how it looks for religious schools. 

In [ ]:
df[df.RELTYPE == 'Religious'].sort_values(by='PTNotFSM6CLA1A', ascending=True)[['SCHNAME', 'TOWN', 'NFTYPE', 'RELDENOM', 'AGERANGE', 'PTNotFSM6CLA1A']]

Low figures for some schools under the PTNotFSM6CLA1A column also suggest some schools (now religious, not secular) with predominantly disadvantaged students. However, to have a more complete picture, let's check some statistics by the same categories (religious and secular) and . For this, we can use the qcut and pivot_table functions.

In [ ]:
sections = pd.cut(df['PTNotFSM6CLA1A'], bins=np.arange(0, 1.1, 0.1))
pd.crosstab(sections, df['RELTYPE'], margins=True)

In [ ]:
len(df) # The total for PTNotFSM6CLA1A is 14560, which is the same as the grand total in the bottom right corner of the table above

The table above shows us that out of 14560 schools, there are 19 where only 10% or less of their students are *not* disadvantaged (4 religious and 15 secular). This is 0.13% of the total. Let's take a look at these schools. 

In [ ]:
df[(df['PTNotFSM6CLA1A'] <= 0.1) & (df['RELTYPE'] == 'Religious')] # Only 4 schools indeed, just as in the table above (religious and with less than 10% of students not disadvantaged)

In [ ]:
df[(df['PTNotFSM6CLA1A'] <= 0.1) & (df['RELTYPE'] == 'Secular')]

In [ ]:
df[(df['PTNotFSM6CLA1A'] <= 0.1) & (df['RELTYPE'] == 'Secular')].shape[0] # 15 schools indeed (secular and with less than 10% of students not disadvantaged)

In [ ]:
pd.crosstab(sections, df['RELTYPE'], margins=True, normalize=True)*100 # To show percentages of each category

In [ ]:
# Notice that here we multipy and divide pandas Series by scalars, which is known as broadcasting
reltype_crosstab = pd.crosstab(sections, df['RELTYPE'], margins=True)
reltype_crosstab['Religious'] = 100 * reltype_crosstab['Religious'] / len(df[df['RELTYPE']=='Religious']) 
reltype_crosstab['Secular'] = 100 * reltype_crosstab['Secular'] / len(df[df['RELTYPE']=='Secular'])
reltype_crosstab['All'] = 100 * reltype_crosstab['All'] / len(df['RELTYPE'])
reltype_crosstab

In [ ]:
reltype_crosstab.iloc[:-1, :].plot(kind='bar', title='Percentage of students not disadvantaged', figsize=(8, 5))

The table and plot above show us that, in general, there are more disadvantaged students in secular schools. However, at the 'higher' end (more than 70% of students are not disadvantaged), religious schools show considerably higher percentages. 

Some spot checks show that these figures are not incorrect. One example is Ward Jackson Church of England VA Primary School in a table above (religious). It is easy to find that it recently had 18 disadvantaged students out of 19 total, which explains why the figure in the PTNotFSM6CLA1A column was so low. Likewise for Seacroft Grange Primary School: 27 disadvantaged pupils out of 30 total.

https://www.compare-school-performance.service.gov.uk/school/136943/ward-jackson-church-of-england-va-primary-school/primary/results-by-pupil-characteristics?accordionstate=0|1

https://www.compare-school-performance.service.gov.uk/school/107928/seacroft-grange-primary-school/primary/results-by-pupil-characteristics?accordionstate=0|1

## School Comparison by School Type

In general, school types are not expected to play an important role in the context of this analysis. While they *are* definitely important on a higher level, it is important to remind that this analysis focuses only on primary education, for younger children, and hence omits a large number of schools which admit only older students. Nevertheless, it may be still helpful to check how schools of different types performed, as this information may be useful for further analysis. 

To start, let's check which schools types are available and what these school type codes actually represent. 

In [ ]:
df['NFTYPE'].unique()

According to the table provided at the beginning of the notebook (Feature Selection section), the codes can be interpreted as below:
AC = Academy Sponsor Led (NFTYPE 20), 
CY = Community school (21), 
VA = Voluntary Aided school (22), 
VC = Voluntary Controlled school (23), 
FD = Foundation school (24), 
CTC = City Technology College (25), 
ACC = Academy converter (51), 
F = Free school (52). Rectype 2: 
CYS = Community special (26), 
FDS = Foundation special (27), 
ACS = Academy special (50), 
FS = Free Special School (53), 
ACCS = Academy Converter Special (55)

Working with the codes may not be the most convenient way to conduct our analysis, and we can replace them. 

In [ ]:
nftype_dict = {
    'ACC': 'Academy converter',
    'CY': 'Community school',
    'AC': 'Academy sponsor-led',
    'VC': 'Voluntary-controlled school',
    'VA': 'Voluntary-aided school',
    'F': 'Free school',
    'FD': 'Foundation school'
}
df.replace({'NFTYPE':nftype_dict}, inplace=True)

In [ ]:
df['NFTYPE'].unique()

Let's check how many schools are in each category

In [ ]:
vc1 = df['NFTYPE'].value_counts(normalize=False)
vc2 = df['NFTYPE'].value_counts(normalize=True) * 100
vc_df = pd.concat([vc1, vc2], axis=1)
vc_df.columns = ['Count', 'Percentage']
vc_df.loc['Total'] = vc_df.sum()
vc_df

In [ ]:
vc_df.iloc[:-1,:].plot.pie(y='Percentage', figsize=(7, 7), autopct='%1.1f%%', legend=False, title='School Types')

Let's run a pivot table to compare median values across the three columns corresponding to KS1 attainment (low, medium, and high respectively)

In [ ]:
schtype_pivot = df.pivot_table(['PTKS1GROUP_H', 'PTKS1GROUP_M', 'PTKS1GROUP_L'], index='NFTYPE', aggfunc=np.median, margins=True)
schtype_pivot

In [ ]:
schtype_pivot.sort_values(by='PTKS1GROUP_H').plot(kind='bar', figsize=(10, 5), rot=45)

The pivot table and bar chart above suggest that all these types of schools (expect, perhaps, academy sponsor-led) are probably not much different. While the medians are, of course, not exactly the same, we do not know yet whether the differences we have seen are statistically significant. Moreover, above we have seen only three features, and we may as well take a look at some others (there might, for example, be some evidence that, say, free schools are better for mathematics). 

As a first step, we can run the same pivot tables as above, but with more features. Later, we will conduct analysis of variance (ANOVA) to check whether the differences are statistically significant.

In [ ]:
cols = ["PTKS1GROUP_L", "PTKS1GROUP_M", "PTKS1GROUP_H", "PTNotFSM6CLA1A",
        "PTRWM_EXP", "PTRWM_HIGH", "PTREAD_EXP", "PTREAD_HIGH", "PTGPS_EXP",
        "PTGPS_HIGH", "PTMAT_EXP", "PTMAT_HIGH", "PTWRITTA_EXP", "PTWRITTA_HIGH",
        "PTSCITA_EXP"]
schtype_pivot = df.pivot_table(cols, index='NFTYPE', aggfunc=np.median, margins=True)
schtype_pivot

Interesting... Looks like most school types are roughly the same, but academy sponsor-led does tend to stand out, and appears to be worse on performance-related features. Note that the table above contains *medians*, which are *not* influenced by outliers. Is this by chance? This appears to be less likely now, and we are going to investigate it further. For now, however, let's repeat our previous excercise and look at violin plots of these features, but this time across school types. 

In [ ]:
df_long = df[cols].stack()
df_long=df_long.to_frame()
df_long.reset_index(inplace=True)
df_long.columns = ['urn', 'category', 'value']
urn_nftype_dict = df['NFTYPE'].to_dict() # Dictionary mapping URN to school type
df_long['nftype'] = df_long['urn'].map(urn_nftype_dict) # Adding a column for school type based on school URN

To make the chart below even more clear, let's replace the codes of features with their long descriptions, in the long dataframe only.

In [ ]:
category_dict = {
    'PTKS1GROUP_L' : 'Percentage of pupils in cohort with low KS1 attainment',
    'PTKS1GROUP_M' : 'Percentage of pupils in cohort with medium KS1 attainment',
    'PTKS1GROUP_H' : 'Percentage of pupils in cohort with high KS1 attainment',
    'PTNotFSM6CLA1A' : 'Percentage of key stage 2 pupils who are not disadvantaged',
    'PTRWM_EXP' : 'Percentage of pupils reaching the expected standard in reading, writing and maths',
    'PTRWM_HIGH' : 'Percentage of pupils achieving a high score in reading and maths and working at greater depth in writing',
    'PTREAD_EXP' : 'Percentage of pupils reaching the expected standard in reading',
    'PTREAD_HIGH' : 'Percentage of pupils achieving a high score in reading',
    'PTGPS_EXP' : 'Percentage of pupils reaching the expected standard in grammar, punctuation and spelling',
    'PTGPS_HIGH' : 'Percentage of pupils achieving a high score in grammar, punctuation and spelling',
    'PTMAT_EXP' : 'Percentage of pupils reaching the expected standard in maths',
    'PTMAT_HIGH' : 'Percentage of pupils achieving a high score in maths',
    'PTWRITTA_EXP' : 'Percentage of pupils reaching the expected standard in writing',
    'PTWRITTA_HIGH' : 'Percentage of pupils working at greater depth within the expected standard in writing',
    'PTSCITA_EXP' : 'Percentage of pupils reaching the expected standard in science TA'
}

df_long.replace({'category':category_dict}, inplace=True)

In [ ]:
type_order = np.sort(df_long.nftype.unique()).tolist() # Unique school types (7 in total) sorted alphabetically
g = sns.FacetGrid(df_long, col='category', col_wrap=1, margin_titles=True, legend_out=True, height=7, aspect=2, sharex=False)
g.map(sns.boxplot, 'nftype', 'value', order=type_order)

Interesting... The plots above strongly indicate that sponsor-led academies tend to have lower performance and higher percentage of disadvantaged pupils. However, is there a chance that these results are not statistically significant and are just a fluke? It is possible, but the fact that we have observed consistently lower performance indicated by several features makes it less likely. Nevertheless, it is still worth it to perform some statistical tests. 

The test we are going to do is the Kruskal-Wallis H-test, which is a non-parametric version of ANOVA (analysis of variance). The null hypothesis is that the population median of all of the groups are equal, and comparing to the 'traditional' ANOVA this test is more flexible. Specifically, it allows for different sizes of the groups, which is exactly the case. It is important to clarify, however, that rejecting the null hypothesis does not indicate which of the groups differs.

Let's start with a simple example. For PTNotFSM6CLA1A (percentage of key stage 2 pupils who are not disadvantaged), we have seen in a plot above that the medians of, for example, for community and foundation schools are identical (both at 0.71 in our samples), but it is very different for sponsor-led academies (only 0.56). The test works for two or more groups, but let's do it for two pairs first, as a 'sanity check'. The two pairs will be: foundation schools vs community schools, and then foundation schools vs sponsor-led academies. In the first case, we expect a failure to reject the null hypothesis (that the medians are equal), and in the second one we expect a strong rejection. 

In [ ]:
x = df['PTNotFSM6CLA1A'][df['NFTYPE'] == 'Foundation school']
y = df['PTNotFSM6CLA1A'][df['NFTYPE'] == 'Community school']
stats.kruskal(x, y)

Indeed, the high p-value indicates that we fail to reject the null hypothesis. 

In [ ]:
x = df['PTNotFSM6CLA1A'][df['NFTYPE'] == 'Foundation school']
y = df['PTNotFSM6CLA1A'][df['NFTYPE'] == 'Academy sponsor-led']
stats.kruskal(x, y)

In line with our expectation, the null hypothesis can be safely rejected even at an extremely low confidence level. 

In the context of this discussion, it is worth reminding what the p-value actually means, since it is often misunderstood and misused. Technically, it is the probability of obtaining test results at least as extreme as the results actually observed, if the null hypothesis is true. In other words, if the true *population* medians are actually equal, p-value is the probability of obtaining *sample* medians at least as extreme as the ones we have actually observed in our samples. 

But there is no need, of course, to limit ourselves only to one feature and two pairs of categories. We can do this for all of them in one go (remember that the Kruskal-Wallis test can be run for two or more arrays, which can be of different lengths). Let's make a function for it, to make it easier to do. 

In [ ]:
cols # Features to test

In [ ]:
type_order # School types in alphabetical order

In [ ]:
def run_kw_tests(sch_types, feature):
    """
    Runs the Kruskal-Wallis test taking each sample in samples_list as a separate input.
    """
    samples_list = []
    for sch_type in sch_types:
        samples_list.append(df[feature][df['NFTYPE'] == sch_type].copy())
    print('-------------------------------')
    print('Running the Kruskal-Wallis test for ' + str(feature))
    print(category_dict[feature])
    print(stats.kruskal(*samples_list, nan_policy='omit'))

In [ ]:
for feature in cols:
    run_kw_tests(type_order, feature)

Unsurprisingly, we have found that there are statistically significant differences between the medians. In other words, the groups by school types *do* come from different populations. 